## Deliverable 3. Create a Travel Itinerary Map.

In [52]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

import numpy as np
from datetime import datetime
import time

In [53]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Alihe,CN,270.60,overcast clouds,50.5667,123.7167,Alihe Forestry Hotel
1,2,Severodvinsk,RU,257.21,overcast clouds,64.5635,39.8302,Park Hotel
2,3,Lebu,CL,286.10,clear sky,-37.6167,-73.6500,Hostal Las Lilas
3,4,Bogatyye Saby,RU,269.31,overcast clouds,56.0102,50.4483,Nur
4,5,Izumo,JP,288.12,scattered clouds,35.3667,132.7667,Toyoko-Inn Izumoshi ekimae


In [54]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [55]:
# 4a. Add a marker layer for each city to the map.
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [56]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df['City']=='Grindavik']
vacation_end = vacation_df.loc[vacation_df['City']=='Grindavik']
vacation_stop1 = vacation_df.loc[vacation_df['City']=='Vestmannaeyjar']
vacation_stop2 = vacation_df.loc[vacation_df['City']=='Hofn']
vacation_stop3 = vacation_df.loc[vacation_df['City']=='Husavik']

vacation_start

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
491,1073,Grindavik,IS,275.41,snow,63.8424,-22.4338,Northern Light Inn & Max's Restaurant


In [57]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
#start = vacation_start[['Lat','Lng']].to_numpy()

start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]

print(start)
print(stop1)
print(stop2)
print(stop3)
print(end)


(63.8424, -22.4338)
(63.4427, -20.2734)
(64.2539, -15.2082)
(66.0449, -17.3389)
(63.8424, -22.4338)


In [58]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

fig = gmaps.figure()
start2end = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3], travel_mode='DRIVING')
fig.add_layer(start2end)
fig

#waypoints=[montreux], travel_mode='BICYCLING'

Figure(layout=FigureLayout(height='420px'))

In [61]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1073,Grindavik,IS,275.41,snow,63.8424,-22.4338,Northern Light Inn & Max's Restaurant
1,430,Vestmannaeyjar,IS,276.43,overcast clouds,63.4427,-20.2734,Glamping & Camping
2,124,Hofn,IS,278.65,light rain,64.2539,-15.2082,Hótel Höfn
3,212,Husavik,IS,264.10,overcast clouds,66.0449,-17.3389,Bjarnabúð


In [62]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [65]:
# 11a. Add a marker layer for each city to the map.

max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))